In [11]:
from elasticsearch import Elasticsearch
import json

# Connect to Elasticsearch
es = Elasticsearch(["http://localhost:9200"], basic_auth=("elastic", "2F8u4KWbdpHG24abCZkd"))

# Query security logs
response = es.search(index="winlogbeat-*", body={"query": {"match_all": {}}})

# Convert response to a dictionary
response_dict = response.body  # Extract the response dictionary

# Save logs to a JSON file
with open("logs.json", "w") as f:
    json.dump(response_dict, f, indent=4)

print("Logs exported successfully!")

Logs exported successfully!


In [12]:
!python convert_logs.py

Converted JSON logs to logs.csv


In [13]:
!python preprocess_graph.py

Graph saved as 'windows_graph.pt'!


In [14]:
!python gnn_model.py

Epoch 0: Loss = 1.5484, Accuracy = 0.3800
Epoch 5: Loss = 0.7109, Accuracy = 0.6300
Epoch 10: Loss = 0.5529, Accuracy = 0.7300
Epoch 15: Loss = 0.4697, Accuracy = 0.7800
Epoch 20: Loss = 0.4354, Accuracy = 0.7900
Epoch 25: Loss = 0.3888, Accuracy = 0.8200
Epoch 30: Loss = 0.3623, Accuracy = 0.8300
Epoch 35: Loss = 0.3297, Accuracy = 0.8700
Epoch 40: Loss = 0.3009, Accuracy = 0.8700
Epoch 45: Loss = 0.2731, Accuracy = 0.8700


In [ ]:
from flask import Flask, request, jsonify
import torch
from gnn_model import ThreatDetectionGNN

app = Flask(__name__)

model = ThreatDetectionGNN(input_dim=128, hidden_dim=64, output_dim=2)
model.load_state_dict(torch.load("gnn_model.pth"))
model.eval()

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    x = torch.tensor(data["features"]).float()
    edge_index = torch.tensor(data["edge_index"]).long()
    with torch.no_grad():
        output = model(x, edge_index)
        prediction = torch.argmax(output, dim=1).tolist()
    return jsonify({"prediction": prediction})

@app.route("/", methods=["GET"])
def home():
    return "Threat Detection API is running!"

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Jun/2025 15:25:51] "POST /predict HTTP/1.1" 200 -
